In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from io import BytesIO
import cv2
import bson
from skimage.data import imread
import matplotlib.pyplot as plt
import keras
from PIL import Image
import io, bson, multiprocessing
import os
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense

Using TensorFlow backend.


In [3]:
def get_the_data(path):
    data = bson.decode_file_iter(open(path, 'rb'))
    images=[]
    category=[]
    for c, d in enumerate(data):
        product_id = d['_id']
        category_id = d['category_id'] 
        for e, pic in enumerate(d['imgs']):
            category.append(category_id)
            im=pic['picture']
            images.append(Image.open(io.BytesIO(im)))
        if(len(set(category))==860):
            break
    return category, images

In [4]:
product_category_train,image_train=get_the_data('E:\\kaggle\\train.bson')

In [5]:
y, rev_labels = pd.factorize(product_category_train)

In [6]:
len(y)

9998

In [7]:
from sklearn.utils import shuffle
im_train,lab_train=shuffle(image_train,y)
test_im=im_train[9980:]
test_lab=lab_train[9980:]
image_train=im_train[:9980]
label_train=lab_train[:9980]

In [8]:
len_train = len(image_train)
len_test = len(test_im)

In [21]:
len_cat = len(image_train)
#
cnt = 0
flag=0
for i in range(len_cat):
        if i == 0:
            n = label_train[i]
            #print('cat', n)
            dir_ = 'C:\\Users\\saish\\Documents\\EE258\\test\\train'
            pathname = os.path.join(dir_, str(n))
            if not os.path.exists(pathname):
                  os.makedirs(pathname)
            os.chdir(pathname)
            d = image_train[i]
            d.save("{0}.jpg".format(str(cnt)))
        if i != 0:      
            if((label_train[i])!=(label_train[i-1])):
                n = label_train[i]
                #print('cat', n)
                dir_ = 'C:\\Users\\saish\\Documents\\EE258\\test\\train'
                pathname = os.path.join(dir_, str(n))  
                if not os.path.exists(pathname):
                      os.makedirs(pathname)
                os.chdir(pathname)
                d = image_train[i]
                d.save("{0}.jpg".format(str(cnt)))
            if((label_train[i])==(label_train[i-1])):
                  n = label_train[i]
                  #print('cat', n)
                  d = image_train[i]
                  d.save("{0}.jpg".format(str(cnt)))
                
        cnt +=1

In [22]:
len_cat = len(test_im)
#
cnt = 0
flag=0
for i in range(len_cat):
        if i == 0:
            n = test_lab[i]
            #print('cat', n)
            dir_ = 'C:\\Users\\saish\\Documents\\EE258\\test\\test1'
            pathname = os.path.join(dir_, str(n))
            if not os.path.exists(pathname):
                  os.makedirs(pathname)
            os.chdir(pathname)
            d = test_im[i]
            d.save("{0}.jpg".format(str(cnt)))
        if i != 0:      
            if((test_lab[i])!=(test_lab[i-1])):
                n = test_lab[i]
                #print('cat', n)
                dir_ = 'C:\\Users\\saish\\Documents\\EE258\\test\\test1'
                pathname = os.path.join(dir_, str(n))  
                if not os.path.exists(pathname):
                      os.makedirs(pathname)
                os.chdir(pathname)
                d = test_im[i]
                d.save("{0}.jpg".format(str(cnt)))
            if((test_lab[i])==(test_lab[i-1])):
                  n = test_lab[i]
                  #print('cat', n)
                  d = test_im[i]
                  d.save("{0}.jpg".format(str(cnt)))
                
        cnt +=1

# KERAS inception

In [8]:
img_width, img_height = 180, 180
top_model_weights_path = 'bottleneck_fc_model_inception.h5'
train_data_dir = 'C:\\Users\\saish\\Documents\\EE258\\test\\train'
validation_data_dir = 'C:\\Users\\saish\\Documents\\EE258\\test\\test'
nb_train_samples = len_train
nb_validation_samples = len_test
epochs = 50
batch_size = 1

In [9]:
def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.inception_v3.InceptionV3(include_top=False, weights='imagenet',input_shape=(180,180,3))

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('bottleneck_features_train_inception.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('bottleneck_features_validation_inception.npy', 'wb'),
            bottleneck_features_validation)

In [10]:
save_bottlebeck_features()

Found 9980 images belonging to 859 classes.
Found 18 images belonging to 10 classes.


In [11]:

train_data = np.load(open('bottleneck_features_train_inception.npy', 'rb'))
train_labels = np.array(label_train)


validation_data = np.load(open('bottleneck_features_validation_inception.npy', 'rb'))
validation_labels = np.array(test_lab)
    

model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(rev_labels), activation='softmax'))

model.compile(optimizer='rmsprop',loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(train_data,train_labels,validation_split=0.2,epochs=2,batch_size=1)
    
model.save_weights(top_model_weights_path)


Train on 7984 samples, validate on 1996 samples
Epoch 1/2
7984/7984 [==============================] - 1629s 204ms/step - loss: 10.7397 - acc: 0.3333 - val_loss: 10.9097 - val_acc: 0.3231
Epoch 2/2
7984/7984 [==============================] - 1000s 125ms/step - loss: 10.7279 - acc: 0.3344 - val_loss: 10.9097 - val_acc: 0.3231


In [9]:
train_data = np.load(open('bottleneck_features_train_inception.npy', 'rb'))
train_labels = np.array(label_train)


validation_data = np.load(open('bottleneck_features_validation_inception.npy', 'rb'))
validation_labels = np.array(test_lab)
    

model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(rev_labels), activation='softmax'))

model.compile(optimizer='rmsprop',loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(train_data,train_labels,validation_split=0.2,epochs=2,batch_size=1)
    
#model.save_weights(top_model_weights_path)

Train on 7984 samples, validate on 1996 samples
Epoch 1/2
7984/7984 [==============================] - 2353s 295ms/step - loss: 14.0982 - acc: 0.1251 - val_loss: 14.1963 - val_acc: 0.1192
Epoch 2/2
7984/7984 [==============================] - 2454s 307ms/step - loss: 14.0935 - acc: 0.1256 - val_loss: 14.1936 - val_acc: 0.1192


In [10]:
train_data = np.load(open('bottleneck_features_train_inception.npy', 'rb'))
train_labels = np.array(label_train)


validation_data = np.load(open('bottleneck_features_validation_inception.npy', 'rb'))
validation_labels = np.array(test_lab)
    

model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(rev_labels), activation='softmax'))

sgd = optimizers.SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer='sgd',loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(train_data,train_labels,validation_split=0.2,epochs=2,batch_size=1)
    
#model.save_weights(top_model_weights_path)

Train on 7984 samples, validate on 1996 samples
Epoch 1/2
7984/7984 [==============================] - 1437s 180ms/step - loss: 16.0104 - acc: 0.0064 - val_loss: 16.1020 - val_acc: 0.0010
Epoch 2/2
7984/7984 [==============================] - 1139s 143ms/step - loss: 16.1111 - acc: 3.7575e-04 - val_loss: 16.0935 - val_acc: 0.0010
